In [ ]:
!apt-get update
!apt-get install libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.0.6 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.0.6.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [ ]:
import gfootball.env as football_env
#import gym # for cartpole (original source)


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    # build the MLP model
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model

    # agent memorises the different policies
    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    # exploration / exploitation
    def act(self, state):
        
        # exploration
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        # exploitation
        # forward pass through the NN, rather than a look-up in Q-value table
        # because this is deep ql
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    # update Q value
    # update the NN
    def replay(self, batch_size):
        
        # take batch from memory
        minibatch = random.sample(self.memory, batch_size)
        
        # for each memory in the batch
        for state, action, reward, next_state, done in minibatch:
            
            # if DONE, make our target reward
            target = reward
            
            if not done:
                # future DISCOUNTED reward
                # use bellman formula
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            
            # make the agent map the current state to future discounted reward (target_f - future discounted reward)
            target_f = self.model.predict(state) # forward run 
            # update according to the observation
            target_f[0][action] = target
            
            # train the MLP model (update the weights)
            # fit(input,labels): input state, predict targe_f (which itself is an estimate)
            self.model.fit(state, target_f, epochs=1, verbose=0)
            
        # decay the epsilon value down to the min value
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [ ]:
env = football_env.create_environment(env_name='academy_empty_goal', representation='simple115', render=False)
    
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
    
done = False
batch_size = 32

# idea is: put the ball in in least number of steps.
# for each EPISODE, if in STEPS we get GOAL then score = 1
# if not, then score = 0
EPISODES = 1000
STEPS = 


# iterate ("play") the game!
for e in range(EPISODES):
    # reset the environment at the beginning of each STEPS-step game
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for t in range(STEPS):
      # decide action
        action = agent.act(state)

      # advance the game to next frame based on the action
        next_state, reward, done, _ = env.step(action)
        
        score = 0
        
        if reward>0:
          print("reward: {}, at time {}".format(reward,t))
          #score = 1
        #  print("time: {}, episode: {}/{}, score: {}, reward: {}, epsilon: {:.2}"
        #          .format(t, e, EPISODES, score, reward, agent.epsilon))
        #  break
        #print("reward: {}, time: {}, episode: {}".format(reward, t, e))
        if done:
          print("reward: {}, at time {}, episode: {}".format(reward,t,e))
          break
        
        next_state = np.reshape(next_state, [1, state_size])

      # memorize the previous state,action,reward and done
        agent.memorize(state, action, reward, next_state, done)

      # make next state the new current state
        state = next_state

        if len(agent.memory) > batch_size:
                    agent.replay(batch_size)